In [210]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
import re
from IPython.display import display
import tensorflow as tf

plt.rc('font', family='Verdana')
%matplotlib inline

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [211]:
sess = tf.InteractiveSession()

In [212]:
width = 28
height = 28
flat = width*height
class_output = 10
padding = 'SAME'
x = tf.placeholder(tf.float32,shape = [None, flat])
y_= tf.placeholder(tf.float32,shape = [None, class_output])

x_image = tf.reshape(x, shape = [-1, width, height, 1])

In [213]:
W_conv1 = tf.Variable(tf.truncated_normal(shape = [5,5,1 ,32], stddev = 0.1))
b_conv1 = tf.Variable(tf.constant(0.1,shape = [32]))
convolve1 = tf.nn.conv2d(x_image, W_conv1, strides = [1,1,1,1], padding = padding) + b_conv1
h_conv1 = tf.nn.relu(convolve1)
conv1 = tf.nn.max_pool(h_conv1, ksize =[1,2,2,1], strides = [1,2,2,1], padding = padding)
#conv1

In [214]:
W_conv2 = tf.Variable(tf.truncated_normal(shape = [5,5,32 ,64], stddev = 0.1))
b_conv2 = tf.Variable(tf.constant(0.1,shape = [64]))
convolve2 = tf.nn.conv2d(conv1, W_conv2,strides = [1,1,1,1], padding = padding) + b_conv2
h_conv2 = tf.nn.relu(convolve2)
conv2 = tf.nn.max_pool(h_conv2,ksize =[1,2,2,1], strides = [1,2,2,1], padding = padding)
#conv2

In [215]:
layer2_matrix = tf.reshape(conv2, [-1,7*7*64])
W_fc1 = tf.Variable(tf.truncated_normal(shape =[7*7*64,1024], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.1,shape = [1024]))
fcl = tf.matmul(layer2_matrix, W_fc1) + b_fc1
h_fcl = tf.nn.relu(fcl)
#h_fcl

keep_prob = tf.placeholder(tf.float32)
layerdrop = tf.nn.dropout(h_fcl,keep_prob)

W_fc2 =  tf.Variable(tf.truncated_normal(shape = [1024,10], stddev = 0.1))
b_fc2 =  tf.Variable(tf.constant(0.1 , shape = [10]))
fc2 = tf.matmul(layerdrop, W_fc2) + b_fc2

y_CNN = tf.nn.softmax(fc2)

In [216]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_CNN), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_CNN,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())

for i in range(20000):
    batch = mnist.train.next_batch(60)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, float(train_accuracy)))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
print("test accuracy %g"% accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.0833333
step 100, training accuracy 0.783333
step 200, training accuracy 0.9
step 300, training accuracy 0.983333
step 400, training accuracy 0.9
step 500, training accuracy 0.966667
step 600, training accuracy 0.95
step 700, training accuracy 0.883333
step 800, training accuracy 0.983333
step 900, training accuracy 0.966667
test accuracy 0.9684
